https://github.com/Berkmann18/facebook-friend-network
に従うことでfacebook friendsのcsv facebook.csvが生成される。
python codeはjupyterからだとうまく動作しなかったので、vs codeから推奨

In [ ]:
library(ggnetwork)

In [ ]:
library(tidyverse)

In [ ]:
library(readr)

In [ ]:
df = read_tsv("two_column_df.tsv")

In [ ]:
df %>% head

In [ ]:
library(igraph)

In [ ]:
run_igraph = graph.data.frame(df %>% select(Source ,Target))

In [ ]:
run_comm = cluster_edge_betweenness(run_igraph)

In [ ]:
class(run_comm)

In [ ]:
membership(run_comm)

In [ ]:
run_igraph = set_vertex_attr(run_igraph , "comm" , value = membership(run_comm))

In [ ]:
run_ggdf = fortify(run_igraph)

In [ ]:
run_ggdf %>% head


In [ ]:
theme_set(theme_void())

In [ ]:

options(repr.plot.width=14, repr.plot.height=10)

In [ ]:
(g = ggplot(run_ggdf , aes(x = x , y = y , xend = xend , yend = yend , color = as.character(comm))) + geom_edges(alpha = 0.2) +
 geom_nodes() + guides(color = F))

http://curleylab.psych.columbia.edu/netviz/netviz1.html#/48

In [ ]:
library(ggiraph)

In [ ]:
gr = g + geom_point_interactive(aes(tooltip = name))

In [ ]:

run_gi = girafe(ggobj = gr , width_svg = 14, height_svg = 10)

In [ ]:
run_gi <- girafe_options(run_gi,
  opts_sizing(rescale = FALSE) )

In [ ]:
run_gi

In [ ]:
?girafe